# TV Script Generation
In this project, I'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  I'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network I'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
I'll be using a subset of the original dataset, consisting of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [28]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Playing around with `view_sentence_range` to view different parts of the data.

In [29]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




### Make sure that tensorflow version is 1.0.0

In [30]:
!pip install -U tensorflow==1.0.0

Requirement already up-to-date: tensorflow==1.0.0 in /Users/kosty/Developer/software/miniconda2/envs/dlnd/lib/python3.6/site-packages
Requirement already up-to-date: protobuf>=3.1.0 in /Users/kosty/Developer/software/miniconda2/envs/dlnd/lib/python3.6/site-packages (from tensorflow==1.0.0)
Requirement already up-to-date: numpy>=1.11.0 in /Users/kosty/Developer/software/miniconda2/envs/dlnd/lib/python3.6/site-packages (from tensorflow==1.0.0)
Requirement already up-to-date: wheel>=0.26 in /Users/kosty/Developer/software/miniconda2/envs/dlnd/lib/python3.6/site-packages (from tensorflow==1.0.0)
Requirement already up-to-date: six>=1.10.0 in /Users/kosty/Developer/software/miniconda2/envs/dlnd/lib/python3.6/site-packages (from tensorflow==1.0.0)
Requirement already up-to-date: setuptools in /Users/kosty/Developer/software/miniconda2/envs/dlnd/lib/python3.6/site-packages (from protobuf>=3.1.0->tensorflow==1.0.0)


## Implement Preprocessing Functions
Implemented the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, first need to transform the words to ids.  In this function, created two dictionaries:
- Dictionary to go from the words to an id, called `vocab_to_int`
- Dictionary to go from the id to word, called `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [31]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    text_counter = Counter(text)
    sorted_text = sorted(text_counter, key=text_counter.get, reverse=True)
    
    vocab_to_int = {word: i for i, word in enumerate(sorted_text)}
    int_to_vocab = {i: word for word, i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
I'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".


In [32]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    token_dict = {
        '.':'||Period||',
        ',':'||Comma||',
        '"':'||Quotation_Mark||',
        ';':'||Semicolon||',
        '!':'||Exclamation_Mark||',
        '?':'||Question_Mark||',
        '(':'||Left_Parentheses||',
        ')':'||Right_Parentheses||',
        '--':'||Dash||',
        '\n':'||Return||'
    }
    return token_dict

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it

In [33]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point

In [34]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
Build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


/Users/kosty/Developer/software/miniconda2/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implemented the `get_inputs()` function to create TF Placeholders for the Neural Network.  
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [36]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float64, name="learning_rate")
    
    return inputs, targets, learning_rate

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [37]:
def get_init_cell(batch_size, rnn_size, num_layers=4):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm for _ in range(num_layers)])
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), 
                                name="initial_state")
    
    return cell, initial_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Applied embedding to `input_data` using TensorFlow.  Returning the embedded sequence.

In [38]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # Completed: Implement Function
    embedded = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedded, input_data)
    
    return embed
                                 

tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
Created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [39]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # Completed: Implement Function
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(state, "final_state")
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [40]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    #"""
    # TODO: Implement Function
    embedded = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embedded)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None, 
                                                weights_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                                               biases_initializer=tf.zeros_initializer())
    return logits, final_state
    #"""



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implemented `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If the last batch can't be filled with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [41]:
import math
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    

    # Get number of batches
    num_batches = len(int_text) // (batch_size * seq_length)
    
    # Create placeholder
    batches = []
    
    for i in range(num_batches):
        inputs = []
        targets = []
        batch = [inputs, targets]
        for ii in range(batch_size):
            start = ii * num_batches * seq_length + i * seq_length
            end = start + seq_length
            inputs.append(int_text[start:end])
            # Check to see if its the end of the sequence
            if ii == (batch_size - 1) and i == (num_batches -1):
                    final = []
                    for iii in range(1, seq_length):
                        final.append(int_text[ii * num_batches * seq_length + i * seq_length + iii])
                    final.append(int_text[0])
                    targets.append(final)   
            else: 
                targets.append(int_text[start+1:end+1])
        batches.append(batch)    
    return np.array(batches)
            
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [64]:
# Number of Epochs
num_epochs = 315
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 212
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph

In [65]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train the Convolutional Net

In [66]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/17   train_loss = 8.822
Epoch   0 Batch   10/17   train_loss = 6.377
Epoch   1 Batch    3/17   train_loss = 6.154
Epoch   1 Batch   13/17   train_loss = 6.010
Epoch   2 Batch    6/17   train_loss = 6.051
Epoch   2 Batch   16/17   train_loss = 5.979
Epoch   3 Batch    9/17   train_loss = 6.012
Epoch   4 Batch    2/17   train_loss = 5.941
Epoch   4 Batch   12/17   train_loss = 5.952
Epoch   5 Batch    5/17   train_loss = 6.002
Epoch   5 Batch   15/17   train_loss = 5.943
Epoch   6 Batch    8/17   train_loss = 5.911
Epoch   7 Batch    1/17   train_loss = 5.890
Epoch   7 Batch   11/17   train_loss = 5.952
Epoch   8 Batch    4/17   train_loss = 5.930
Epoch   8 Batch   14/17   train_loss = 5.916
Epoch   9 Batch    7/17   train_loss = 5.952
Epoch  10 Batch    0/17   train_loss = 5.934
Epoch  10 Batch   10/17   train_loss = 5.927
Epoch  11 Batch    3/17   train_loss = 5.955
Epoch  11 Batch   13/17   train_loss = 5.913
Epoch  12 Batch    6/17   train_loss = 5.959
Epoch  12 

Epoch 107 Batch   11/17   train_loss = 1.784
Epoch 108 Batch    4/17   train_loss = 1.824
Epoch 108 Batch   14/17   train_loss = 1.777
Epoch 109 Batch    7/17   train_loss = 1.765
Epoch 110 Batch    0/17   train_loss = 1.655
Epoch 110 Batch   10/17   train_loss = 1.637
Epoch 111 Batch    3/17   train_loss = 1.656
Epoch 111 Batch   13/17   train_loss = 1.568
Epoch 112 Batch    6/17   train_loss = 1.712
Epoch 112 Batch   16/17   train_loss = 1.666
Epoch 113 Batch    9/17   train_loss = 1.583
Epoch 114 Batch    2/17   train_loss = 1.682
Epoch 114 Batch   12/17   train_loss = 1.620
Epoch 115 Batch    5/17   train_loss = 1.607
Epoch 115 Batch   15/17   train_loss = 1.614
Epoch 116 Batch    8/17   train_loss = 1.626
Epoch 117 Batch    1/17   train_loss = 1.756
Epoch 117 Batch   11/17   train_loss = 1.658
Epoch 118 Batch    4/17   train_loss = 1.786
Epoch 118 Batch   14/17   train_loss = 1.922
Epoch 119 Batch    7/17   train_loss = 1.876
Epoch 120 Batch    0/17   train_loss = 1.858
Epoch 120 

Epoch 215 Batch    5/17   train_loss = 0.386
Epoch 215 Batch   15/17   train_loss = 0.407
Epoch 216 Batch    8/17   train_loss = 0.401
Epoch 217 Batch    1/17   train_loss = 0.494
Epoch 217 Batch   11/17   train_loss = 0.447
Epoch 218 Batch    4/17   train_loss = 0.467
Epoch 218 Batch   14/17   train_loss = 0.497
Epoch 219 Batch    7/17   train_loss = 0.464
Epoch 220 Batch    0/17   train_loss = 0.456
Epoch 220 Batch   10/17   train_loss = 0.467
Epoch 221 Batch    3/17   train_loss = 0.449
Epoch 221 Batch   13/17   train_loss = 0.429
Epoch 222 Batch    6/17   train_loss = 0.474
Epoch 222 Batch   16/17   train_loss = 0.464
Epoch 223 Batch    9/17   train_loss = 0.417
Epoch 224 Batch    2/17   train_loss = 0.431
Epoch 224 Batch   12/17   train_loss = 0.439
Epoch 225 Batch    5/17   train_loss = 0.402
Epoch 225 Batch   15/17   train_loss = 0.429
Epoch 226 Batch    8/17   train_loss = 0.407
Epoch 227 Batch    1/17   train_loss = 0.483
Epoch 227 Batch   11/17   train_loss = 0.434
Epoch 228 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [67]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [68]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [69]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # Completed: Implement Function
    input = loaded_graph.get_tensor_by_name("input:0")
    initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state = loaded_graph.get_tensor_by_name("final_state:0")
    probabilities = loaded_graph.get_tensor_by_name("probs:0")
    return input, initial_state, final_state, probabilities


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [70]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # Completed: Implement Function
    return np.random.choice(list(int_to_vocab.values()), p=probabilities)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script, set `gen_length` to the length of TV script you want to generate.

In [71]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(defensive) you made some good friends, didn't ya?
lenny_leonard: just some worst party.
moe_szyslak: what are you should fun.
sam: homer, i know it? everything crappy like this today, moe.
moe_szyslak: whoa, hey, do you look over to the situation from.


moe_szyslak: well, i can't do that what we been pick far the home for that.
lenny_leonard: even the same woman.
jacques: oh, these strap of düffenbraus, in the kind and come?


lenny_leonard: all got to help that the guy in so old blue face" moe" here back here.
carl_carlson:(sighs) i didn't told that milk with, i'm doing to turn down tonight.
homer_simpson:(admitting) then if you fold no in your bad more-- let me look down," when by all another joint. i'm, lenny's you seen me back?
chief_wiggum: ooo! that up here, my new surprised.
chief_wiggum: you know, what if i was, let's


# The TV Script is Nonsensical
The TV script doesn't make any sense, because it was trained on less than a megabyte of text.  In order to get good results, we have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Project Outcome
This net works pretty well on generating new text. 